## Part 1: Creating Your Training Examples

First, let’s create some examples that will help your assistant learn. We’ll use a friendly tool that makes this process easier. Create a new notebook called Legal_Assistant_Training and add this cell:

In [ ]:
import json
import os
from IPython.display import display, clear_output
import ipywidgets as widgets

class ExampleCreator:
    """A friendly tool to help create training examples"""
    def __init__(self):
        self.examples = []
        self.filename = 'training_data.jsonl'
        # Create our input boxes
        self.question_box = widgets.Textarea(
            description='Question:',
            placeholder='Type your legal question here...',
            layout={'width': '90%', 'height': '100px'}
        )
        self.answer_box = widgets.Textarea(
            description='Answer:',
            placeholder='Type the correct answer here...',
            layout={'width': '90%', 'height': '200px'}
        )
        # Create our buttons
        self.save_button = widgets.Button(description='Save Example')
        self.save_button.on_click(self.save_example)
        self.display_button = widgets.Button(description='Show All Examples')
        self.display_button.on_click(self.show_examples)
        # Show our tool
        display(self.question_box)
        display(self.answer_box)
        display(self.save_button)
        display(self.display_button)
    def save_example(self, button):
        """Save a new example"""
        question = self.question_box.value.strip()
        answer = self.answer_box.value.strip()
        if not question or not answer:
            print("❌ Please provide both a question and an answer!")
            return
        # Create the example
        example = {
            "conversations": [
                {"from": "human", "value": question},
                {"from": "gpt", "value": answer}
            ]
        }
        # Save it
        with open(self.filename, 'a') as f:
            f.write(json.dumps(example) + '\n')
        self.examples.append(example)
        # Clear the boxes for next example
        self.question_box.value = ''
        self.answer_box.value = ''
        print(f"✅ Example saved! You now have {len(self.examples)} examples.")
    def show_examples(self, button):
        """Show all saved examples"""
        clear_output(wait=True)
        print(f"Your {len(self.examples)} Training Examples:\n")
        for i, example in enumerate(self.examples, 1):
            print(f"Example {i}:")
            print(f"Q: {example['conversations'][0]['value']}")
            print(f"A: {example['conversations'][1]['value']}\n")
        # Show our tool again
        display(self.question_box)
        display(self.answer_box)
        display(self.save_button)
        display(self.display_button)
# Create our example creator tool
creator = ExampleCreator()
print("✨ Example Creator is ready! Start adding your training examples above.")

Let’s add some starter examples to help you understand the format. Add this cell:

In [1]:
# Some example legal questions and answers to get you started
import json

starter_examples = [
    {
        "question": "What makes a contract valid?",
        "answer": "A valid contract requires four essential elements: 1) Offer and acceptance, "
                  "2) Consideration (something of value exchanged), 3) Intention to create "
                  "legal relations, and 4) Capacity of the parties to contract. The agreement "
                  "must also be legal and sufficiently certain in its terms."
    },
    {
        "question": "How does force majeure work in contracts?",
        "answer": "Force majeure clauses excuse a party from performing their contractual "
                  "obligations when extraordinary events beyond their control prevent "
                  "performance. These events typically include natural disasters, wars, or "
                  "government actions. The clause must specifically define what constitutes "
                  "force majeure, and the party claiming it must prove the event's impact."
    }
]
# Add these examples using our tool
for example in starter_examples:
    with open('training_data.jsonl', 'a') as f:
        formatted_example = {
            "conversations": [
                {"from": "human", "value": example["question"]},
                {"from": "gpt", "value": example["answer"]}
            ]
        }
        f.write(json.dumps(formatted_example) + '\n')
print("✅ Added starter examples to your training data!")

✅ Added starter examples to your training data!


Now you can use the tool to add your own examples. Here are some tips for creating good examples:

    Include different types of questions:
    “What is…” (definitions)
    “How does…” (processes)
    “Why is…” (reasoning)
    “When should…” (timing)

Make sure your answers:

    Start with a clear main point
    Include specific details
    Use proper legal terminology
    Stay concise but complete

Try to create at least 50–70 examples before moving on to the next part. The more, the better!

## Part 2: Fine-Tuning and Deploying Your Model

Once you have your training examples ready, you can use them to improve your model. This part is more technical and requires a new Colab notebook with more computational resources.

Create a new notebook called Legal_Assistant_Finetuning and add these cells:
Step 1: Install Dependencies

### Step 1: Install Dependencies

In [ ]:
# Install required packages
!pip install -qqq "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" --progress-bar off
from torch import __version__
from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install -qqq --no-deps {xformers} trl peft accelerate bitsandbytes triton --progress-bar off

### Step 2: Import Libraries and Load Model

In [ ]:
import torch
from trl import SFTTrainer
from datasets import load_dataset
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported

# Load model
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)
# Prepare model for PEFT
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth"
)

### Step 3: Prepare Data and Tokenizer

In [ ]:
# Set up tokenizer with chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="chatml",
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"}
)
def apply_template(examples):
    messages = examples["conversations"]
    text = [tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=False) 
            for message in messages]
    return {"text": text}

# Load and prepare dataset
dataset = load_dataset("json", data_files='training_data.jsonl')
dataset = dataset.map(apply_template, batched=True)


### Step 4: Configure Training

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset['train'],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        learning_rate=3e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        num_train_epochs=1,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=10,
        output_dir="output",
        seed=0,
    ),
)

# Start training
trainer.train()

### Step 5: Save and Test the Model

In [ ]:
# Prepare model for inference
model = FastLanguageModel.for_inference(model)

# Test with a sample query
messages = [
    {"from": "human", "value": "What is your purpose?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")
# Generate response
text_streamer = TextStreamer(tokenizer)
outputs = model.generate(
    input_ids=inputs, 
    streamer=text_streamer, 
    max_new_tokens=128, 
    use_cache=True
)
# Save the model
model.save_pretrained_merged("model", tokenizer, save_method="merged_16bit")
# Optional: Save GGUF versions
model.save_pretrained_gguf("model", tokenizer, "q8_0")
quant_methods = ["q2_k", "q3_k_m", "q4_k_m", "q5_k_m", "q6_k", "q8_0"]
for quant in quant_methods:
    model.save_pretrained_gguf(f"model_{quant}", tokenizer, quant)